Here is something in Julia

In [ ]:
println("Hello julia!")

Now, here is a for loop in Julia

In [ ]:
for i in 1:1_000_000
    println(i)
end

Now, we start working with `Tidier`

In [3]:
# Load the required libraries
using RDatasets
using Tidier
using Chain

We will work with the `movies` dataset

In [6]:
movies = dataset("ggplot2", "movies");

In [7]:
# Check the first 10 lines
first(movies, 10)

Row,Title,Year,Length,Budget,Rating,Votes,R1,R2,R3,R4,R5,R6,R7,R8,R9,R10,MPAA,Action,Animation,Comedy,Drama,Documentary,Romance,Short
,String,Int32,Int32,Int32?,Float64,Int32,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Cat…,Int32,Int32,Int32,Int32,Int32,Int32,Int32
1,$,1971,121,missing,6.4,348,4.5,4.5,4.5,4.5,14.5,24.5,24.5,14.5,4.5,4.5,,0,0,1,1,0,0,0
2,$1000 a Touchdown,1939,71,missing,6.0,20,0.0,14.5,4.5,24.5,14.5,14.5,14.5,4.5,4.5,14.5,,0,0,1,0,0,0,0
3,$21 a Day Once a Month,1941,7,missing,8.2,5,0.0,0.0,0.0,0.0,0.0,24.5,0.0,44.5,24.5,24.5,,0,1,0,0,0,0,1
4,"$40,000",1996,70,missing,8.2,6,14.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,34.5,45.5,,0,0,1,0,0,0,0
5,"$50,000 Climax Show, The",1975,71,missing,3.4,17,24.5,4.5,0.0,14.5,14.5,4.5,0.0,0.0,0.0,24.5,,0,0,0,0,0,0,0
6,$pent,2000,91,missing,4.3,45,4.5,4.5,4.5,14.5,14.5,14.5,4.5,4.5,14.5,14.5,,0,0,0,1,0,0,0
7,$windle,2002,93,missing,5.3,200,4.5,0.0,4.5,4.5,24.5,24.5,14.5,4.5,4.5,14.5,R,1,0,0,1,0,0,0
8,'15',2002,25,missing,6.7,24,4.5,4.5,4.5,4.5,4.5,14.5,14.5,14.5,4.5,14.5,,0,0,0,0,1,0,1
9,'38,1987,97,missing,6.6,18,4.5,4.5,4.5,0.0,0.0,0.0,34.5,14.5,4.5,24.5,,0,0,0,1,0,0,0


In [8]:
# Describe the dataset
describe(movies, :nmissing)

Row,variable,nmissing
,Symbol,Int64
1,Title,0
2,Year,0
3,Length,0
4,Budget,53573
5,Rating,0
6,Votes,0
7,R1,0
8,R2,0
9,R3,0


We find the movies with budget

In [9]:
movies_with_budget = @chain movies begin
    @filter(Budget != ismissing(Budget))
end

Row,Title,Year,Length,Budget,Rating,Votes,R1,R2,R3,R4,R5,R6,R7,R8,R9,R10,MPAA,Action,Animation,Comedy,Drama,Documentary,Romance,Short
,String,Int32,Int32,Int32?,Float64,Int32,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Cat…,Int32,Int32,Int32,Int32,Int32,Int32,Int32
1,'G' Men,1935,85,450000,7.2,281,0.0,4.5,4.5,4.5,4.5,14.5,34.5,34.5,4.5,4.5,,0,0,0,1,0,0,0
2,'Manos' the Hands of Fate,1966,74,19000,1.6,7996,74.5,4.5,4.5,4.5,4.5,4.5,4.5,4.5,4.5,14.5,,0,0,0,0,0,0,0
3,'Til There Was You,1997,113,23000000,4.8,799,4.5,4.5,4.5,14.5,14.5,14.5,14.5,4.5,4.5,14.5,PG-13,0,0,1,0,0,1,0
4,.com for Murder,2002,96,5000000,3.7,271,64.5,4.5,4.5,4.5,4.5,4.5,4.5,4.5,4.5,4.5,,0,0,0,0,0,0,0
5,10 Things I Hate About You,1999,97,16000000,6.7,19095,4.5,4.5,4.5,4.5,4.5,14.5,24.5,14.5,14.5,14.5,PG-13,0,0,1,0,0,1,0
6,100 Mile Rule,2002,98,1100000,5.6,181,4.5,4.5,4.5,4.5,14.5,24.5,14.5,14.5,4.5,14.5,R,0,0,1,0,0,0,0
7,100 Proof,1997,94,140000,3.3,19,14.5,14.5,4.5,14.5,14.5,14.5,14.5,0.0,0.0,24.5,,0,0,0,1,0,0,0
8,101,1989,117,200000,7.8,299,4.5,0.0,4.5,4.5,4.5,4.5,4.5,14.5,14.5,45.5,,0,0,0,0,1,0,0
9,101-vy kilometer,2001,103,200000,5.8,7,0.0,0.0,14.5,0.0,0.0,44.5,0.0,14.5,0.0,24.5,,0,0,0,1,0,0,0


In [10]:
# Describe the movies_with_budget dataset
describe(movies_with_budget, :nmissing) # We observe that there is no missing budget now for any of the movies

Row,variable,nmissing
,Symbol,Int64
1,Title,0
2,Year,0
3,Length,0
4,Budget,0
5,Rating,0
6,Votes,0
7,R1,0
8,R2,0
9,R3,0


We report how many movies with budget exist in the dataset

In [19]:
(nrow, ncol) = size(movies_with_budget)
println("There are $nrow movies with budget in the \"movies\" dataset")

There are 5183 movies with budget in the "movies" dataset


Now, we select certain columns for movies that meet certain criteria, as described in the `Tidier` documentation:

In [20]:
select_movies = @chain movies_with_budget begin
    @mutate(Budget_in_mil = Budget /1_000_000)
    @filter(Budget >= mean(Budget))
    @select(Title, Budget_in_mil)
    @slice(1:10)
end

Row,Title,Budget_in_mil
,String,Float64
1,'Til There Was You,23.0
2,10 Things I Hate About You,16.0
3,102 Dalmatians,85.0
4,13 Going On 30,37.0
5,"13th Warrior, The",85.0
6,1492: Conquest of Paradise,47.0
7,15 Minutes,42.0
8,1941,35.0
9,2 Fast 2 Furious,76.0


In [21]:
# We find the top 10 movies based on Budget
top10_budget = @chain movies_with_budget begin
    @mutate(Budget_in_mil = Budget /1_000_000)
    @arrange(desc(Budget_in_mil))
    @select(Title, Budget_in_mil)
    @slice(1:10)
end

Row,Title,Budget_in_mil
,String,Float64
1,Spider-Man 2,200.0
2,Titanic,200.0
3,Troy,185.0
4,Terminator 3: Rise of the Machines,175.0
5,Waterworld,175.0
6,Wild Wild West,170.0
7,Van Helsing,160.0
8,Alexander,150.0
9,Master and Commander: The Far Side of the World,150.0


In [22]:
# We select everything related to "Alexander"
alexander_movie = @chain movies_with_budget begin
    @filter(Title == "Alexander")
end

Row,Title,Year,Length,Budget,Rating,Votes,R1,R2,R3,R4,R5,R6,R7,R8,R9,R10,MPAA,Action,Animation,Comedy,Drama,Documentary,Romance,Short
,String,Int32,Int32,Int32?,Float64,Int32,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Cat…,Int32,Int32,Int32,Int32,Int32,Int32,Int32
1,Alexander,2004,175,150000000,5.5,11254,14.5,4.5,4.5,4.5,14.5,14.5,14.5,4.5,4.5,14.5,R,0,0,0,1,0,0,0
